# 【RAG+Tools】基于ErnieBot打造企业级Agent应用

----
### 项目背景


Ernie-bot等生成式人工智能在文本生成和文本到图像生成等任务中表现出令人印象深刻的性能。然而，这些模型也存在固有的局限性，包括产生幻觉、缺乏对生成文本的可解释性、对专业领域知识理解不足，以及对最新知识的了解有限。为了克服这些限制并提高模型的能力，有两种主要途径：一是通过微调（Fine Tune）更新模型，另一种是让它们能够与外部世界互动，以不同形式和方式获取知识。

**微调虽然有效** ，可以让模型真正“学会”一些私域知识，但也会带来几个问题。首先，由于生成模型依赖于内在知识（权重），它们仍然无法完全避免产生幻觉。在需要高度准确性的场景中，这是不可接受的，因为用户很难从表面上判断模型是否在胡说八道。其次，在现实场景中，每时每刻都在产生大量数据，对事物的概念迭代飞快，如政策解读或指标调整等。而模型微调并不是一个简单的工作，无论是从数据准备、算力资源、微调效果还是训练时间等各方面来看，随时用新数据进行微调都是不现实的。即便能够做到每月更新一次，已经是非常理想的状态，但最终的微调效果也无法保证。

另一种解决方案是**检索增强生成（RAG）**，它为生成式模型提供了与外部世界互动的前景。RAG的主要作用类似搜索引擎，找到与用户提问最相关的知识或对话历史，并结合原始提问（查询），生成信息丰富的提示，指导模型生成准确输出。其本质上应用了情境学习（In-Context Learning）的原理。

![RAG原理](https://starpaddle.bj.bcebos.com/RAG.png)



### 项目演示
![](https://starpaddle.bj.bcebos.com/xmys.gif)


### 环境安装

**本文所用工具**
- 文档解析：extract_office_content, rapidocr_pdf, rapidocr_onnxruntime
- 提取特征向量：ErnieBot-Text-Embedding
- 向量存储：sqlite
- 向量检索：faiss
- 大语言模型：Ernie-bot

In [1]:
!pip install -r requirements.txt --user

Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 490.7 kB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 538.3 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 230.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 233.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 321.6 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 439.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 562.4 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.6/339.6 kB 589.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 563.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 517.9 kB/s eta 0:00:0000:0100

### 配置文件说明

**配置文件路径** ： /home/aistudio/config.yaml
- title: 显示在web端首页的名字（建议用企业or项目名称命名）
- sidebar_title：侧边栏Title
- version：显示在web端首页的版本号
- access_token：访问令牌（用于调用大模型能力时的身份验证） 获取地址：https://aistudio.baidu.com/index/accessToken
- SENTENCE_SIZE: 切分句子的最大长度
- upload_dir: 知识库存储地址
- question：预设问题（供用户一键体验使用）
- DEFAULT_PROMPT：预设Prompt（可根据输出需求和角色定义进行修改）

### 准备知识库

1. 将准备好的文本知识库，放到存储位置：/home/aistudio/assets/raw_upload_files
2. 运行Knowledge.py，进行语义向量的提取与存储

**程序工作流程**
1. 加载目录知识库
2. 自动根据文件类型（pdf\txt）来提取文件内容
3. 内容的文本分块
    - 使用多个正则表达式将文本分割为初步句子。这些正则表达式处理不同的标点符号（如句号、问号、感叹号、省略号等），并在这些符号后添加换行符，以标识句子分界。
    - 去掉文本末尾多余的换行符。
    - 遍历初步分割的句子列表，如果某个句子长度超过设定的最大长度，再次进行细分。
        - 句子长度由config.yaml中的SENTENCE_SIZE决定。
    - 使用正则表达式处理不同的标点符号（如逗号）进行更细的分割。
    - 如果细分后的句子仍然过长，再进一步细分，处理一些特定的标点符号和空格。
    - 将所有处理后的句子加入到最终的句子列表中，并返回该列表。
4. 将文件内容分批次传递给ErnieEncodeText实例进行向量提取。
    - 每批次的大小由config.yaml中的encoder_batch_size决定。
5. 提取到的向量和原始内容一起存储到向量数据库中。
    - 每个文件的数据存储时会生成一个唯一的UID，用于后续查询。
    
**使用说明**
1. 查看数据库中的数据：

- 输入view，然后提供需要查询的数据UID。
2. 清空数据库：

- 输入clear，将清空整个向量数据库。
3. 提取向量：

- 输入extract，从指定目录中加载文件，提取向量，并存储到数据库中。

In [1]:
# 语义向量提取
!python knowledge_extract.py

['## ERNIE SDK 简介', 'ERNIE SDK 提供了一系列工具和接口，帮助开发者快速构建和部署基于 ERNIE 模型的应用。', '详细的使用指南可以参考 [ERNIE SDK 使用文档](https://ernie-bot-agent.readthedocs.io/zh-cn/latest/)。', '## 如何调用文心一言 API', '调用文心一言 API 的详细步骤可以参考 [文心一言 API 使用文档](https://ernie-bot-agent.readthedocs.io/zh-cn/latest/)。', '## 基于文心大模型的能力开发应用', '可以前往飞桨星河社区进行应用开发，平台支持零代码开发和 Notebook 开发两种方式。', '详细教程请参考 [应用开发教程合集](https://aistudio.baidu.com/topic/tutorial)。', '若有进一步开发需求请联系飞桨星河社区的工作人员。', '## 访问令牌/Token Key 获取', 'Token Key 地址： [获取 Token Key](https://aistudio.baidu.com/usercenter/token)', '## Token 包兑换与购买', '- **兑换 Token 包**：星河社区首页右上角控制台 -> 点击 Token 余额 -> 兑换 Token -> 输入兑换码', '- **购买 Token**：星河社区首页右上角控制台 -> 点击 Token 余额 -> 购买 Token -> 扫码支付', '## 应用部署与发布', '1. 星河社区首页左侧边栏选择“应用” -> 我创建的 -> 找到待发布的应用 -> 右上角点击发布。']
['2. 发布时可以设置封面图、应用名称，以及选择是否公开源项目。', '## 前端静态资源部署', 'AI Studio 现已支持前端静态资源部署功能，开发者可以在项目中自由编辑定制开发前端代码，开发后可一键将静态资源包部署为公网服务。', '申请地址：[前端静态资源部署功能](https://aistudio.baidu.com/activitydetail/1503019310)。', '## 问题与解决方案', '- **服务访问过于频繁**：增加 `time sl

### 自定义Tools

**Tools**，犹如大模型的“手”“脚”。单单依靠文本生成功能并不能全面满足企业的需求，这时我们需要扩展大模型的能力边界，让大模型拥有更多的技能，而这些技能就是我们所说的“Tools”。具体的应用案例包括：

1. **数据处理和分析**：大模型可以使用专门的工具来处理和分析大量数据，帮助企业进行精准的市场分析、客户需求预测和趋势分析，从而制定更有效的战略决策。

2. **自动化客服**：通过集成语音识别、情感分析和自然语言理解工具，大模型可以提供高效、准确的自动化客服，解决客户问题，提高客户满意度。

3. **内容生成与审核**：大模型可以使用文本生成和审核工具，帮助企业快速生成高质量的营销内容、新闻稿和技术文档，并通过审核工具确保内容的准确性和合规性。

4. **图像和视频处理**：大模型结合图像识别和视频处理工具，可以自动化处理和分析视觉数据，应用于质量检测、安防监控和媒体内容制作等领域。

5. **语言翻译与本地化**：通过集成多语言翻译和本地化工具，大模型可以帮助企业在全球市场中更好地沟通和运营，提供多语言支持和本地化服务。

通过引入和集成各种工具，我们可以大幅度扩展大模型的功能和应用范围，满足企业在不同场景下的多样化需求。这不仅提升了大模型的实际应用价值，也为企业提供了更多创新和发展的机会。


**本文将基于Ernie-sdk，打造一个具有灵活性，可拓展性的RAG&Tools企业级应用框架**


![](https://starpaddle.bj.bcebos.com/Agent.png)

In [ ]:
def translation(text):
    url = 'http://tool-translation.aistudio-hub.baidu.com/translation'

    q = text  # example: hello
    from_lang = 'zh'  # example: en
    to_lang = 'en'  # example: zh
    term_ids = ''  # 术语库id，多个逗号隔开
    headers = {'Content-Type': 'application/json', 'Authorization': f'token {tools_token}'}
    payload = {'q': q, 'from': from_lang, 'to': to_lang, 'termIds': term_ids}
    r = requests.post(url, json=payload, headers=headers)
    result = r.json()
    result = result['result']['trans_result'][0]['dst']
    return str(result)


# 第一个类是描述这个工具输入的参数
class TranslationInputView(ToolParameterView):
    text: str = Field(description="需要翻译的文本")


# 第二个类就是描述response这个变量，这是这个工具输出的参数。
class TranslationOutputView(ToolParameterView):
    response: str = Field(description="已经翻译好的文本")


class Translation(Tool):
    description: str = "Texttospeech，是一个文本翻译工具。"
    input_type: Type[ToolParameterView] = TranslationInputView
    output_type: Type[ToolParameterView] = TranslationOutputView

    async def __call__(self, text: str) -> Dict[str, str]:
        response = await asyncio.to_thread(translation, text)
        return {"response": response}


- 可参照例子进行自定义其它工具，使大模型拥有更多的能力
- 飞桨提供了很多的工具接口：https://aistudio.baidu.com/application/center/tool
    - 大家可以选择喜欢的工具，查阅其文档，在该项目中实现工具的添加
- 添加工具后，请在config.yaml文件中Tools下进行参数的新增和description的描述
    - 具体可参考config,yaml中Translation工具的书写规范

### Streamlit应用部署

本文开头所示的Streamlit应用已经打包在根目录下的main.streamlit.py文件内，如知识库向量已经提取成功，大家可按照aistudio的部署方法进行部署

具体步骤如下：

1. 编辑器右上角`部署`按钮  
![](https://starpaddle.bj.bcebos.com/bs.png)
2. 选择`Streamlit`应用

![](https://starpaddle.bj.bcebos.com/22222816.png)

3. 填写应用信息，`Streamlit`选择最新的即可，`Repo`填写唯一的名称（可看作是该应用的ID）

![](https://starpaddle.bj.bcebos.com/streamlit.png)

### Streamlit应用发布

1. 部署成功后，点击进入应用空间，点击右上角`发布应用`

![](https://starpaddle.bj.bcebos.com/fbyy.png)

2. 点击`添加版本`自定义版本号，执行文件选择`main.streamlit.py`，运行资源选择`CPU套餐`，开启`公开发布`

![](https://starpaddle.bj.bcebos.com/3327.png)

### 总结

本文详细阐述了如何以Ernie-bot为基础，构建企业级RAG-Agent应用。通过RAG与各类工具的紧密配合，我们不仅能够确保大模型在回答问题时能够引入最新、最准确的知识，还能有效利用这些工具拓展大模型的能力范围，从而更好地满足企业多样化、深层次的需求。文章深入剖析了RAG的工作原理以及工具所带来的增值能力，同时还详细介绍了知识库数据的准备、文本的切块处理、向量数据库的启动以及应用部署等一系列关键步骤。最后，我们通过检索数据和Agent的推理思考，充分验证了该框架的实用性和有效性。此方案为企业提供了一种切实可行的大语言模型应用落地解决方案，助力大语言模型在特定行业或专业领域应对深入应用的挑战。